In [2]:
lib = r'/srv01/technion/shitay/Code/classifying_response_to_immunotherapy/'
import sys
sys.path.append(lib)
from utilities.package_importing import *
import scipy.stats as stats

# NATMI conversions
Here you can convert expression matrix for NATMI

# Option 1 - Immune clusters:

In [143]:
COHORT_PATH = r'/storage/md_keren/shitay/Data/droplet_seq/M97_M173/subcohort/normalized/1.1.22/subcohort_normalized_1.1.22_protein_coding_genes.pkl'
IMMUNE_CLUSTER_BARCODE_MAPPING_PATH = r'/storage/md_keren/shitay/outputs/clustering/immune/summary/subcohort_1.1.22_run_1.1.22/subcohort_immune_1.1.22_clusters_mapping.csv'
immune_annotation_path = r'/storage/md_keren/shitay/Data/tables/clustering_annotations/subcohort_1.1.22/17.1.22/immune.xlsx'

FOLDER_PATH = r'/storage/md_keren/shitay/outputs/NATMI/immune/conversions/20.3.22'

In [144]:
cohort = pickle.load(open(COHORT_PATH, 'rb'))
clusters_barcodes_mapping_df = pd.read_csv(IMMUNE_CLUSTER_BARCODE_MAPPING_PATH)
immune_annotation_df = pd.read_excel(immune_annotation_path, engine='openpyxl').set_index('CLUSTER_IDX')
immune_cohort=cohort.get_subset_by_identifiers(clusters_barcodes_mapping_df.Sample, clusters_barcodes_mapping_df.Barcode)

Annotations df:

In [145]:
clusters_barcodes_mapping_df['cell'] =  clusters_barcodes_mapping_df.Sample +'_'+ clusters_barcodes_mapping_df.Barcode
clusters_barcodes_mapping_df['annotation'] = clusters_barcodes_mapping_df['Cluster'].apply(lambda x: immune_annotation_df.loc[x])

In [146]:
clusters_barcodes_mapping_df[['cell', 'annotation']].set_index('cell').to_csv(f'{FOLDER_PATH}/ann.csv',index=True,header=True)

Expression df:

In [148]:
pd.DataFrame(immune_cohort.counts.T, columns=[k+'_'+v for k,v in zip(immune_cohort.samples,immune_cohort.barcodes)], index=immune_cohort.features).to_csv(f'{FOLDER_PATH}/count_table.csv',index=True,header=True)

# Option 2 - tumor GEPs & CD8 & myeloid clusters:

In [3]:
# extract cohort
COHORT_PATH = r'/storage/md_keren/shitay/Data/droplet_seq/M97_M173/subcohort/normalized/16.3.22/subcohort_normalized_1.1.22_protein_coding_genes.pkl'

cohort = pickle.load(open(COHORT_PATH, 'rb'))

In [4]:
CD8_CLUSTER_BARCODE_MAPPING_PATH = r'/storage/md_keren/shitay/outputs/clustering/CD8/summary/subcohort_1.1.22_run_1.1.22/subcohort_CD8_1.1.22_clusters_mapping.csv'
MYELOID_CLUSTER_BARCODE_MAPPING_PATH = r'/storage/md_keren/shitay/outputs/clustering/myeloid/summary/subcohort_1.1.22_run_1.1.22/subcohort_myeloid_1.1.22_clusters_mapping.csv'

myeloid_annotation_path = r'/storage/md_keren/shitay/Data/tables/clustering_annotations/subcohort_1.1.22/17.1.22/Myeloid.xlsx'
CD8_annotation_path = r'/storage/md_keren/shitay/Data/tables/clustering_annotations/subcohort_1.1.22/17.1.22/CD8.xlsx'


# Extract barcode mapping of myeloid & CD8 clusters
CD8_clusters_barcodes_mapping_df = pd.read_csv(CD8_CLUSTER_BARCODE_MAPPING_PATH)
myeloid_clusters_barcodes_mapping_df = pd.read_csv(MYELOID_CLUSTER_BARCODE_MAPPING_PATH)

# Extract myeloid & CD8 cluster annotations
myeloid_annotation_df = pd.read_excel(myeloid_annotation_path, engine='openpyxl').set_index('cluster')
CD8_annotation_df = pd.read_excel(CD8_annotation_path, engine='openpyxl').set_index('cluster')

In [5]:
melanoma_clinical_data = get_constant_cohort(3) #get_clinical_data(71)#ICI=True, after_biopsy='ICI')
# melanoma_clinical_data

GEP_DF, R_GEP_DF, NR_GEP_DF = get_GEP_dfs(melanoma_clinical_data, EXEC_DIR = r'/storage/md_keren/shitay/outputs/cNMF/executions/tumor_runs/subcohort_1.1.22',
                    RUN_RANGE = '20', selected_K=20, number_of_genes=2000, n_replicates=200, local_density_threshold='0.10', FEATURE='response', FEATURE_A='R', FEATURE_B='NR')

filtered_GEP_DF = filter_contaminated_cells_out_of_GEP_DF(GEP_DF, 14, contaminated_cells_path = r'/storage/md_keren/shitay/Data/tables/GEP/subcohort_1.1.22/contaminated_cells_list/GEP14_contaminated_cells.csv')

Using clinical table in path:
 /storage/md_keren/shitay/Data/tables/Melanoma_clinical_data_12.21_unportected.csv

and labels:
/storage/md_keren/shitay/Data/tables/clinical_labels.csv


NATMI annotation df build:

In [11]:
# columns rename
GEP_NATMI_df = filtered_GEP_DF[['sample', 'barcode', 'associated program']].rename(columns={'associated program': 'cluster'})
CD8_NATMI_df = CD8_clusters_barcodes_mapping_df[['Sample', 'Barcode', 'Cluster']].rename(columns={'Sample': 'sample', 'Barcode': 'barcode', 'Cluster': 'cluster'})
myeloid_NATMI_df = myeloid_clusters_barcodes_mapping_df[['Sample', 'Barcode', 'Cluster']].rename(columns={'Sample': 'sample', 'Barcode': 'barcode', 'Cluster': 'cluster'})

# fill cluster annotations
myeloid_NATMI_df['annotation'] = "Myeloid_cluster_"+ myeloid_NATMI_df['cluster'].astype(str) #myeloid_NATMI_df['cluster'].apply(lambda x: myeloid_annotation_df.loc[x])
CD8_NATMI_df['annotation'] = "CD8_cluster_"+ CD8_NATMI_df['cluster'].astype(str) #CD8_NATMI_df['cluster'].apply(lambda x: CD8_annotation_df.loc[x])
GEP_NATMI_df['annotation'] = "GEP_#"+GEP_NATMI_df['cluster'].astype(str)

# remove cells defined as both myeloid and CD8
CD8_ids = [tuple(a) for a in CD8_NATMI_df[['sample', 'barcode']].values.tolist()]
myeloid_ids = [tuple(a) for a in myeloid_NATMI_df[['sample', 'barcode']].values.tolist()]

CD8_NATMI_df = CD8_NATMI_df[[not id in myeloid_ids for id in CD8_ids]]
myeloid_NATMI_df = myeloid_NATMI_df[[not id in CD8_ids for id in myeloid_ids]]


# concatanate dfs
NATMI_df = GEP_NATMI_df.append(CD8_NATMI_df).append(myeloid_NATMI_df)

# set id as the connection of sample and barcode
NATMI_df['cell'] = NATMI_df['sample'] + '_' + NATMI_df['barcode']

save NATMI annotation df:

In [14]:
NATMI_ann_df = NATMI_df[['cell', 'annotation']].set_index('cell')
NATMI_ann_df.to_csv('/storage/md_keren/shitay/outputs/NATMI/tumor_CD8_myeloid/conversions/21.3.22/ann.csv',index=True,header=True)

Expression df build:

In [15]:
NATMI_cohort = cohort.get_subset_by_identifiers(NATMI_df['sample'], NATMI_df['barcode'])

In [16]:
NATMI_exp_matrix_df = pd.DataFrame(NATMI_cohort.counts.T, columns=[k+'_'+v for k,v in zip(NATMI_cohort.samples,NATMI_cohort.barcodes)], index=NATMI_cohort.features)
NATMI_exp_matrix_df.to_csv('/storage/md_keren/shitay/outputs/NATMI/tumor_CD8_myeloid/conversions/21.3.22/count_table.csv',index=True,header=True)

# DEBUG

In [191]:
CD8_CLUSTER_BARCODE_MAPPING_PATH = r'/storage/md_keren/shitay/outputs/clustering/CD8/summary/subcohort_1.1.22_run_1.1.22/subcohort_CD8_1.1.22_clusters_mapping.csv'
MYELOID_CLUSTER_BARCODE_MAPPING_PATH = r'/storage/md_keren/shitay/outputs/clustering/myeloid/summary/subcohort_1.1.22_run_1.1.22/subcohort_myeloid_1.1.22_clusters_mapping.csv'

myeloid_annotation_path = r'/storage/md_keren/shitay/Data/tables/clustering_annotations/subcohort_1.1.22/17.1.22/Myeloid.xlsx'
CD8_annotation_path = r'/storage/md_keren/shitay/Data/tables/clustering_annotations/subcohort_1.1.22/17.1.22/CD8.xlsx'


# Extract barcode mapping of myeloid & CD8 clusters
CD8_clusters_barcodes_mapping_df = pd.read_csv(CD8_CLUSTER_BARCODE_MAPPING_PATH)
myeloid_clusters_barcodes_mapping_df = pd.read_csv(MYELOID_CLUSTER_BARCODE_MAPPING_PATH)

In [208]:
CD8_ids = [tuple(a) for a in CD8_NATMI_df[['sample', 'barcode']].values.tolist()]
myeloid_ids = [tuple(a) for a in myeloid_NATMI_df[['sample', 'barcode']].values.tolist()]

In [213]:
# sum([id in myeloid_ids for id in CD8_ids])
sum([not id in CD8_ids for id in myeloid_ids])

7609

In [217]:
len(myeloid_NATMI_df)

7609

In [203]:
Counter([a[0] for a in inter])

Counter({'M111': 8,
         'M137': 1,
         'M146': 9,
         'M151': 4,
         'M136': 2,
         'M162': 4,
         'M145': 4,
         'M147': 4})

In [207]:
Counter([k.split('_')[0] for k,v in dict(Counter(l)).items() if v>1])

Counter({'M111': 8,
         'M136': 2,
         'M145': 4,
         'M146': 9,
         'M162': 4,
         'M147': 4,
         'M151': 4,
         'M137': 1})

In [149]:
ddd = pd.read_csv('/storage/md_keren/shitay/outputs/NATMI/tumor_CD8_myeloid/conversions/20.3.22/count_table.csv')

In [157]:
em = ddd.set_index('Unnamed: 0')

In [158]:
em.columns

Index(['M106_AAACCTGAGATCCCAT-1', 'M106_AAACCTGCACTAAGTC-1',
       'M106_AAACCTGGTTGTCTTT-1', 'M106_AAACCTGTCATGCATG-1',
       'M106_AAACCTGTCGTATCAG-1', 'M106_AAACGGGAGGCCCGTT-1',
       'M106_AAACGGGCATGTAAGA-1', 'M106_AAACGGGTCGCTTAGA-1',
       'M106_AAAGATGCATTTGCTT-1', 'M106_AAAGATGTCCAGATCA-1',
       ...
       'M131_TTAGGCATCAGAGCTT-1', 'M131_TTAGTTCAGATGTGTA-1',
       'M131_TTAGTTCTCTTCCTTC-1', 'M131_TTATGCTAGTGCTGCC-1',
       'M131_TTGAACGCACCAGCAC-1', 'M131_TTGCGTCAGAGCTATA-1',
       'M146_AGGGTGATCACCACCT-1', 'M147_GGGACCTAGGGTTCCC-1',
       'M147_GGGCATCAGCTATGCT-1', 'M147_TGCACCTGTAAGTGTA-1'],
      dtype='object', length=67885)

In [159]:
ann = pd.read_csv('/storage/md_keren/shitay/outputs/NATMI/tumor_CD8_myeloid/conversions/20.3.22/ann.csv', index_col=0, header=0)

In [161]:
len(set(em.columns).intersection(set(ann.index)))

67849

In [162]:
len(set(em.columns).intersection(set(ann.index)))

67849

In [163]:
len(set(em.columns))

67885

In [164]:
len(set(em.columns).intersection(set(ann.index)))

67849

In [182]:
if len(set(em.columns)) == len(em.columns):
    print(len(em.columns))
if len(set(ann.index)) != len(ann.index):
    print(len(ann.index))

67885
67885


In [190]:
l = ann.index.tolist()
{k:v for k,v in dict(Counter(l)).items() if v>1}

{'M111_CGTTAGACAATCGGTT-1': 2,
 'M111_TTGCCGTGTCGAGTTT-1': 2,
 'M136_ACTGAACGTTGAGTTC-1': 2,
 'M136_GGACAAGGTCAGAAGC-1': 2,
 'M145_ATGCGATAGAGGGATA-1': 2,
 'M146_ACTGAGTTCCCTTGTG-1': 2,
 'M162_AACGTTGAGTGACATA-1': 2,
 'M111_ACACTGAAGCGATAGC-1': 2,
 'M111_ATGGGAGAGCACGCCT-1': 2,
 'M111_CACCAGGCACACCGAC-1': 2,
 'M111_CAGCTGGGTATTCGTG-1': 2,
 'M111_CTAGTGATCTCTGCTG-1': 2,
 'M111_TTCGGTCAGGCCCTTG-1': 2,
 'M147_CTTAGGAAGTGCTGCC-1': 2,
 'M151_CACACCTGTCTGATCA-1': 2,
 'M151_GATGAGGTCAACCAAC-1': 2,
 'M145_GACTAACCAGTCACTA-1': 2,
 'M145_GGACAGATCCTCTAGC-1': 2,
 'M146_CTCGGAGAGTACTTGC-1': 2,
 'M146_GACGGCTTCACCTCGT-1': 2,
 'M162_CCTACCATCGCACTCT-1': 2,
 'M162_GTTCATTTCTCGTATT-1': 2,
 'M146_ACACTGAGTCGAAAGC-1': 2,
 'M146_CCATGTCCAGCCAATT-1': 2,
 'M137_TGAGGGAAGTATCTCG-1': 2,
 'M145_AGTGAGGGTTGGTGGA-1': 2,
 'M146_CCCAATCAGCTAGCCC-1': 2,
 'M146_CCGGGATTCTTGAGAC-1': 2,
 'M146_GAACGGACACATTAGC-1': 2,
 'M146_GAGTCCGGTTCAGTAC-1': 2,
 'M147_CATGACAGTCCGACGT-1': 2,
 'M147_CCAGCGATCAGCGACC-1': 2,
 'M147_G

In [127]:
ids = [k+'_'+v for k,v in zip(NATMI_cohort.samples,NATMI_cohort.barcodes)]

In [129]:
len(ids)

67885

In [141]:
# NATMI_df['cell']#.set_index('cell')
# Counter(NATMI_df['annotation'])

are_the_lists_identical(ids, NATMI_df['cell'].tolist())

True

In [142]:
NATMI_df[['cell', 'annotation']].set_index('cell')

,annotation
cell,
M106_AAACCTGAGATCCCAT-1,GEP #15
M106_AAACCTGCACTAAGTC-1,GEP #17
M106_AAACCTGGTTGTCTTT-1,GEP #15
M106_AAACCTGTCATGCATG-1,GEP #17
M106_AAACCTGTCGTATCAG-1,GEP #17
...,...
M131_TTGCGTCAGAGCTATA-1,Mac-PLTP+MACRO+RBP4+ and DC3- FSCN1+
M146_AGGGTGATCACCACCT-1,Mac-PLTP+MACRO+RBP4+ and DC3- FSCN1+
M147_GGGACCTAGGGTTCCC-1,Mac-PLTP+MACRO+RBP4+ and DC3- FSCN1+


In [125]:
pd.DataFrame(NATMI_cohort.counts.T, columns=[k+'_'+v for k,v in zip(NATMI_cohort.samples,NATMI_cohort.barcodes)], index=NATMI_cohort.features)

,M106_AAACCTGAGATCCCAT-1,M106_AAACCTGCACTAAGTC-1,M106_AAACCTGGTTGTCTTT-1,M106_AAACCTGTCATGCATG-1,M106_AAACCTGTCGTATCAG-1,M106_AAACGGGAGGCCCGTT-1,M106_AAACGGGCATGTAAGA-1,M106_AAACGGGTCGCTTAGA-1,M106_AAAGATGCATTTGCTT-1,M106_AAAGATGTCCAGATCA-1,...,M131_TTAGGCATCAGAGCTT-1,M131_TTAGTTCAGATGTGTA-1,M131_TTAGTTCTCTTCCTTC-1,M131_TTATGCTAGTGCTGCC-1,M131_TTGAACGCACCAGCAC-1,M131_TTGCGTCAGAGCTATA-1,M146_AGGGTGATCACCACCT-1,M147_GGGACCTAGGGTTCCC-1,M147_GGGCATCAGCTATGCT-1,M147_TGCACCTGTAAGTGTA-1
ENSG00000000003,2.436964,1.794323,0.0,0.0,0.0,0.487696,0.0,0.00000,0.659384,0.875673,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
ENSG00000000005,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.00000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
ENSG00000000419,2.436964,0.000000,0.0,0.0,0.0,0.000000,0.0,0.80492,0.000000,0.875673,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
ENSG00000000457,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.00000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.072502
ENSG00000000460,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.00000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000273331,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.00000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
ENSG00000273395,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.00000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
ENSG00000273398,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.00000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
ENSG00000273439,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.00000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [9]:
Counter(GEP_DF['associated program'])

Counter({15: 2697,
         17: 5966,
         14: 874,
         4: 3132,
         3: 231,
         8: 182,
         10: 699,
         18: 5929,
         6: 3857,
         13: 3027,
         9: 18,
         12: 7776,
         20: 16,
         7: 109,
         11: 101,
         16: 1224,
         19: 665,
         5: 11095,
         2: 3732,
         1: 71})